# TẠO CÁC TẬP DỮ LIỆU TRAIN, TEST (SPLITS)

1. Yêu cầu chung: Tạo ra các splits, mỗi split tương ứng với một tập dữ liệu train - test
  + Bài học lý thuyết (để trả lời cho các câu hỏi như vì sao phải cần việc này, thực hiện việc này như thế nào):
    - https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation
    - https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/
    - https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

  <IMG SRC = 'https://raw.githubusercontent.com/satishgunjal/images/master/KFold_Cross_Validation.png'>
2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Yamaha, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/12WrC9APRzQX36cataTccBiN_Z5F2C0yD
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước https://colab.research.google.com/drive/1sZBm78OiTUOqbg5-Z2UOKlk3B37TAgd8
    + Số splits NumSplits - mặc định NumSplits=5 (tương đương 5-fold CV)
- Output:
    + File MotocycleDataset.csv - Tập tin chứa tất cả ảnh của dataset
      - Chương trình sẽ scan qua cây thư mục để tìm tất cả các ảnh (chỉ chọn các ảnh có định dạng + phần mở rộng là .jpg)
      - Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
            - ImageFullPath ở dạng <Thư mục Hiệu xe>/<Tên ảnh>. Ví dụ: Honda/2024123.Honda.1.jpg.
            - CategoryID là số nguyên thuộc [0..5] theo quy ước
              - 0: Others
              - 1: Honda
              - 2: Suzuki
              - 3: Yamaha
              - 4: VinFast     
    + File MotocycleDataset-Splits-[1..5]-[Train/Test].csv - Phân chia thành các splits, mỗi split gồm các ảnh được chia thành thành 2 tập Train - Test
      + Chương trình sẽ đọc dữ liệu từ file  MotocycleDataset.csv, sau đó với mỗi hiệu xe, chia ngẫu nhiên thành 5 tập dữ liệu. Lưu ý là phải chia theo hiệu xe, để đảm bảo dữ liệu Train/Test có dữ liệu của các hiệu xe.
      + Từ 5 tập dữ liệu chia ngẫu nhiên theo các hiệu xe Xij (i là thứ tự tập dữ liệu, j là CategoryID), gom lại thành 5 tập dữ liệu lớn hơn Xi, sao cho mỗi tập dữ liệu này chứa đủ dữ liệu của tất cả các hiệu xe. Tức là Xi = Union(Xij)       
      + Từ 5 tập dữ liệu Xi này (tương ứng với FOLDi ở trong hình vẽ trên), tạo ra 5 splits Split-i
      + Với mỗi bộ dữ liệu Split-i, ghi xuống thành 2 tập tin tương ứng với Train, Test. Ví dụ Split-1 thì ghi thành tập tin  MotocycleDataset-Splits-1-Train.csv và MotocycleDataset-Splits-1-Test.csv. Tập Train gồm 4 bộ, Test gồm bộ còn lại. Ví dụ,
          + Split-1: tập Train sẽ gồm X2, X3, X4, X5, tập Test là X1
          + Split-5: tập Train sẽ gồm X1, X2, X3, X4, tập Test là X5
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
- Lưu ý:
  - Nên viết thêm các cell
    - Hiển thị danh sách các tên tập tin ảnh trong từng Split-Train/Test,
    - Thống kê các ảnh cho từng CategoryID trong mỗi Split-Train/Test
  - Cần có chú thích
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - 10/06/2024

4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.

In [ ]:
# Import Libraries
import os
import re
from sklearn.model_selection import StratifiedKFold
import csv
import pandas as pd
from collections import defaultdict
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Hàm tạo file MotocycleDataset.csv
def create_dataset_csv(root_folder, output_folder):
    """Tạo tệp CSV chứa thông tin về tất cả các ảnh."""
    output_file = os.path.join(output_folder, "MotocycleDataset.csv")
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(["ImageFullPath", "CategoryID"])  # Header

        category_map = {"Others": 0, "Honda": 1, "Suzuki": 2, "Yamaha": 3, "VinFast": 4}

        for category in category_map:
            category_folder = os.path.join(root_folder, category)
            for filename in os.listdir(category_folder):
                  image_path = os.path.join(category, filename)
                  writer.writerow([image_path, category_map[category]])



In [ ]:
# Hàm split các file Train và Test
def create_split_csvs(dataset_file, output_folder,  num_splits=5):
    """Tạo các tệp CSV cho Train/Test của từng split."""

    data = []  # Lưu trữ đường dẫn
    labels = []  # Lưu trữ các nhãn (CategoryID)
    with open(dataset_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Bỏ qua header
        for row in reader:
            data.append(row[0]) # Lưu trữ đường dẫn
            labels.append(row[1])  # Lưu trữ nhãn

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
    for fold, (train_indices, test_indices) in enumerate(skf.split(data, labels), start=1):
        train_file = os.path.join(output_folder, f"MotocycleDataset-Splits-{fold}-Train.csv")
        test_file = os.path.join(output_folder, f"MotocycleDataset-Splits-{fold}-Test.csv")

        with open(train_file, 'w', newline='') as train_csv, open(test_file, 'w', newline='') as test_csv:
            train_writer = csv.writer(train_csv)
            test_writer = csv.writer(test_csv)
            train_writer.writerow(["ImageFullPath", "CategoryID"])
            test_writer.writerow(["ImageFullPath", "CategoryID"])

            # Get the train and test images for this fold
            train_images = [data[i] for i in train_indices]
            train_labels = [labels[i] for i in train_indices]
            test_images = [data[i] for i in test_indices]
            test_labels = [labels[i] for i in test_indices]

            # Write the data to the CSV files
            for image, category in zip(train_images, train_labels):
                train_writer.writerow([image, category])
            for image, category in zip(test_images, test_labels):
                test_writer.writerow([image, category])

In [ ]:
# --- Sử dụng ---
root_folder = "/content/drive/MyDrive/Dataset/Public"
output_folder = "/content/drive/MyDrive/Dataset"

create_dataset_csv(root_folder, output_folder)
pd.read_csv(os.path.join(output_folder, "MotocycleDataset.csv"))


,ImageFullPath,CategoryID
0,Others/22521705-22520777-22520202-22521304-225...,0
1,Others/22521705-22520777-22520202-22521304-225...,0
2,Others/22521705-22520777-22520202-22521304-225...,0
3,Others/22521705-22520777-22520202-22521304-225...,0
4,Others/22521705-22520777-22520202-22521304-225...,0
...,...,...
30216,VinFast/22520896-22520926-22521627.VinFast.165...,4
30217,VinFast/22520896-22520926-22521627.VinFast.159...,4
30218,VinFast/22520896-22520926-22521627.VinFast.172...,4
30219,VinFast/22520896-22520926-22521627.VinFast.167...,4


In [ ]:
dataset_file = "/content/drive/MyDrive/Dataset/MotocycleDataset.csv"
create_split_csvs(dataset_file, output_folder)
for fold in range(1, 6):
    for data_type in ["Train", "Test"]:
        file_name = f"MotocycleDataset-Splits-{fold}-{data_type}.csv"
        file_path = os.path.join(output_folder, file_name)

        # Hiển thị danh sách tên tập tin
        df = pd.read_csv(file_path)
        print(file_name)
        print(df)

        # Thống kê số lượng ảnh cho từng CategoryID
        print("\nThống kê CategoryID:")
        print(df['CategoryID'].value_counts())

MotocycleDataset-Splits-1-Train.csv
                                           ImageFullPath  CategoryID
0      Others/22521705-22520777-22520202-22521304-225...           0
1      Others/22521705-22520777-22520202-22521304-225...           0
2      Others/22521705-22520777-22520202-22521304-225...           0
3      Others/22521705-22520777-22520202-22521304-225...           0
4      Others/22521705-22520777-22520202-22521304-225...           0
...                                                  ...         ...
24171  VinFast/22520896-22520926-22521627.VinFast.96.jpg           4
24172  VinFast/22520896-22520926-22521627.VinFast.160...           4
24173  VinFast/22520896-22520926-22521627.VinFast.165...           4
24174  VinFast/22520896-22520926-22521627.VinFast.159...           4
24175  VinFast/22520896-22520926-22521627.VinFast.161...           4

[24176 rows x 2 columns]

Thống kê CategoryID:
CategoryID
1    6969
3    5947
2    4483
0    4444
4    2333
Name: count, dtype: int64
M